# Data Processing using pyspark

This Notebook covers the algorithms for

- Extraction: Extracting features from “raw” data
- Transformation: Scaling, converting, or modifying features
- Selection: Selecting a subset from a larger set of features
- Locality Sensitive Hashing (LSH): This class of algorithms combines aspects of feature transformation with other algorithms.

In [1]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName('Dataprocessing using pyspark').getOrCreate()

## 1. Feature Extraction

### 1.1 TF- IDF [Term Frequency - Inverse Document Frequency ] 
Feature Vectorization Technique method used for data mining to reflect the importance of term to a document in the corpus.

Denote a term by `t`, a document by `d`, and the corpus by `D`. Term frequency `TF(t,d)` is the number of times that term `t` appears in document `d`, while document frequency `DF(t,D)` is the number of documents that contains term `t`.

\begin{equation}
    IDF(t,d) = log \frac{|D|+1}{DF(t,D)+1}
\end{equation}

where `|D|` is the total number of documents in the corpus

\begin{equation}
TFIDF(t,d,D)=TF(t,d)⋅IDF(t,D).
\end{equation}


**TF** : HashingTF and CountVectorizer can be used for Term Frequency

`HashingTF` is a Transformer which takes sets of terms and converts those sets into fixed-length feature vectors.
HashingTF utilizes the hashing trick. A raw feature is mapped into an index (term) by applying a hash function. The hash function used here is MurmurHash 3.

`CountVectorizer` converts text documents to vectors of term counts

In [2]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])
sentenceData.show(truncate=False)

print("Tokenize the sentence ")
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
word_data = tokenizer.transform(sentenceData)
word_data.show(truncate=False)

print("Transform to TF - HashingTF ")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurized_data = hashingTF.transform(word_data)
featurized_data.show(truncate=False)

print("Transform to TF - CountVectorizer ")
cv = CountVectorizer(inputCol="words", outputCol="rawFeatures1")
c_featurized_data = cv.fit(word_data).transform(word_data)
c_featurized_data.show(truncate=False)

+-----+-----------------------------------+
|label|sentence                           |
+-----+-----------------------------------+
|0.0  |Hi I heard about Spark             |
|0.0  |I wish Java could use case classes |
|1.0  |Logistic regression models are neat|
+-----+-----------------------------------+

Tokenize the sentence 
+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+

Transform to TF - HashingTF 
+-----+----------------------

**IDF** : IDF is an Estimator which is fit on a dataset and produces an `IDFModel`. The `IDFModel` takes feature vectors (generally created from `HashingTF` or `CountVectorizer`) and scales each feature. Intuitively, it down-weights features which appear frequently in a corpus.



In [3]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurized_data)
rescaledData = idfModel.transform(featurized_data)

rescaledData.select("label", "features").show(truncate=False)

+-----+-------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                   |
+-----+-------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(20,[6,8,13,16],[0.28768207245178085,0.6931471805599453,0.28768207245178085,0.5753641449035617])                                           |
|0.0  |(20,[0,2,7,13,15,16],[0.6931471805599453,0.6931471805599453,1.3862943611198906,0.28768207245178085,0.6931471805599453,0.28768207245178085])|
|1.0  |(20,[3,4,6,11,19],[0.6931471805599453,0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453])                       |
+-----+---------------------------------------------------------------------------------------------------------

## 1.2 Word2Vec

`Word2Vec` is an Estimator which takes sequences of words representing documents and trains a `Word2VecModel`. 
- The model maps each word to a unique fixed-size vector. 
- The `Word2VecModel` transforms each document into a vector using the average of all words in the document; 
- This vector can then be used as features for prediction, document similarity calculations, etc.
- Computes distributed vector representation of words

In [4]:
from pyspark.ml.feature import Word2Vec
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])

# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)

result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))


Text: [Hi, I, heard, about, Spark] => 
Vector: [-0.024519025813788176,-0.025680204108357432,-0.010560354497283698]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [-0.0755529555359057,0.04371664673089981,-0.08695207749094282]

Text: [Logistic, regression, models, are, neat] => 
Vector: [-0.03579759057611227,0.012968631088733674,-0.0072596315294504166]



TypeError: Column is not iterable